In [67]:
from sympy import *
from sympy.vector import CoordSys3D, Del
from sympy.simplify.epathtools import EPath

In [2]:
R = CoordSys3D('R')
delop = Del()
x = R.x
y = R.y

eps, theta, r = symbols('epsilon theta r')
nx = Function('n_x')(x, y)
ny = Function('n_y')(x, y)

In [3]:
n = nx * R.i + ny * R.j
n

(n_x(R.x, R.y))*R.i + (n_y(R.x, R.y))*R.j

### Calculate charge density

In [4]:
splay = simplify( delop.dot(n).doit()**2 )
splay

(Derivative(n_x(R.x, R.y), R.x) + Derivative(n_y(R.x, R.y), R.y))**2

In [5]:
bend = simplify( n.cross( delop.cross(n).doit() ).magnitude()**2 )
bend = bend.subs(nx**2 + ny**2, 1)
bend

(Derivative(n_x(R.x, R.y), R.y) - Derivative(n_y(R.x, R.y), R.x))**2

In [6]:
g = (1 + eps) * splay + (1 - eps) * bend
g

(1 - epsilon)*(Derivative(n_x(R.x, R.y), R.y) - Derivative(n_y(R.x, R.y), R.x))**2 + (epsilon + 1)*(Derivative(n_x(R.x, R.y), R.x) + Derivative(n_y(R.x, R.y), R.y))**2

### Calculate in terms of director angle

In [7]:
phi = Function('phi')(x, y)
phi

phi(R.x, R.y)

In [8]:
g = simplify( g.subs({nx: cos(phi), ny: sin(phi)}) )
g

(1 - epsilon)*(sin(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.y) + cos(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x))**2 + (epsilon + 1)*(-sin(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x) + cos(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.y))**2

### Change to [polar coordinates](https://www.wikiwand.com/en/Polar_coordinate_system#Differential_calculus)

In [9]:
phi_p = Function('phi')(theta)

In [10]:
dphi_dx = cos(theta) * phi_p.diff(r) - (1/r) * sin(theta) * phi_p.diff(theta)
dphi_dy = sin(theta) * phi_p.diff(r) + (1/r) * cos(theta) * phi_p.diff(theta)

In [11]:
g_polar = g.subs({phi.diff(x) : dphi_dx, phi.diff(y) : dphi_dy, phi : phi_p})
g_polar = simplify(g_polar)
display(g_polar)

((1 - epsilon)*sin(theta - phi(theta))**2 + (epsilon + 1)*cos(theta - phi(theta))**2)*Derivative(phi(theta), theta)**2/r**2

### Plug it into [Euler-Lagrange equation](https://www.wikiwand.com/en/Euler%E2%80%93Lagrange_equation#Single_function_of_several_variables_with_single_derivative)

In [12]:
euler_lagrange = simplify(g_polar.diff(phi_p) - g_polar.diff(phi_p.diff(theta)).diff(theta))
euler_lagrange = trigsimp(factor(euler_lagrange, eps))
euler_lagrange = simplify(euler_lagrange * -Rational(1, 2) * r**2)
euler_lagrange = factor(euler_lagrange, eps, phi_p.diff(theta, 2), sin(2*theta - 2*phi_p))
display(euler_lagrange)

epsilon*(Derivative(phi(theta), theta)**2 - 2*Derivative(phi(theta), theta))*sin(2*theta - 2*phi(theta)) + epsilon*cos(2*theta - 2*phi(theta))*Derivative(phi(theta), (theta, 2)) + Derivative(phi(theta), (theta, 2))

### Assume ansatz

In [13]:
from sympy.simplify.fu import *

In [14]:
q, C = symbols('q C')
ansatz = q * theta + eps * C * sin(2 * (1 - q) * theta)

In [15]:
ansatz_euler_lagrange = simplify(euler_lagrange.subs(phi_p, ansatz))
ansatz_euler_lagrange = expand(ansatz_euler_lagrange).subs({eps**2: 0, eps**3: 0})
display(ansatz_euler_lagrange)

4*C*epsilon*q**2*sin(2*q*theta - 2*theta) - 8*C*epsilon*q*sin(2*q*theta - 2*theta) + 4*C*epsilon*sin(2*q*theta - 2*theta) + epsilon*q**2*sin(2*C*epsilon*sin(2*q*theta - 2*theta) - 2*q*theta + 2*theta) - 2*epsilon*q*sin(2*C*epsilon*sin(2*q*theta - 2*theta) - 2*q*theta + 2*theta)

In [16]:
sin_subs = {sin(2*C*eps*sin(2*q*theta - 2*theta)
                - 2*q*theta + 2*theta):
                sin(2*C*eps*sin(2*q*theta - 2*theta))
                * cos(2*theta - 2*q*theta)
                +
                sin(2*theta - 2*q*theta)
                * cos(2*C*eps*sin(2*q*theta - 2*theta))}

ansatz_euler_lagrange = ansatz_euler_lagrange.subs(sin_subs)
display(ansatz_euler_lagrange)

4*C*epsilon*q**2*sin(2*q*theta - 2*theta) - 8*C*epsilon*q*sin(2*q*theta - 2*theta) + 4*C*epsilon*sin(2*q*theta - 2*theta) + epsilon*q**2*(sin(2*C*epsilon*sin(2*q*theta - 2*theta))*cos(2*q*theta - 2*theta) - sin(2*q*theta - 2*theta)*cos(2*C*epsilon*sin(2*q*theta - 2*theta))) - 2*epsilon*q*(sin(2*C*epsilon*sin(2*q*theta - 2*theta))*cos(2*q*theta - 2*theta) - sin(2*q*theta - 2*theta)*cos(2*C*epsilon*sin(2*q*theta - 2*theta)))

In [17]:
trig_subs = {sin(2*C*eps*sin(2*q*theta - 2*theta)):
             2*C*eps*sin(2*q*theta - 2*theta),
             cos(2*C*eps*sin(2*q*theta - 2*theta)):
             1}
                 
ansatz_euler_lagrange = expand(ansatz_euler_lagrange.subs(trig_subs))
ansatz_euler_lagrange = ansatz_euler_lagrange.subs(eps**2, 0)
display(ansatz_euler_lagrange)

4*C*epsilon*q**2*sin(2*q*theta - 2*theta) - 8*C*epsilon*q*sin(2*q*theta - 2*theta) + 4*C*epsilon*sin(2*q*theta - 2*theta) - epsilon*q**2*sin(2*q*theta - 2*theta) + 2*epsilon*q*sin(2*q*theta - 2*theta)

In [18]:
sol = solve(ansatz_euler_lagrange, C)
display(sol[0])

q*(q - 2)/(4*(q**2 - 2*q + 1))

In [19]:
single_defect_approximation = ansatz.subs(C, sol[0])
display(single_defect_approximation)

epsilon*q*(q - 2)*sin(theta*(2 - 2*q))/(4*(q**2 - 2*q + 1)) + q*theta

### Above is the first-order approximation to a single defect under the Dzyaloshinskii solution

## Two-defect approximation

### First, Euler Lagrange in cartesian coordinates

In [20]:
euler_lagrange = -simplify(g.diff(phi) 
                           - (g.diff(phi.diff(x)).diff(x) 
                              + g.diff(phi.diff(y)).diff(y))) / 2
euler_lagrange = factor(euler_lagrange, phi.diff(x, 2), phi.diff(y, 2))
euler_lagrange = trigsimp(euler_lagrange)
euler_lagrange = simplify(euler_lagrange)
euler_lagrange = simplify(TR7(TR5(euler_lagrange)))
euler_lagrange = factor(euler_lagrange, sin(2*phi), cos(2*phi), eps)
display(euler_lagrange)

epsilon*(-2*Derivative(phi(R.x, R.y), R.x)*Derivative(phi(R.x, R.y), R.y) - Derivative(phi(R.x, R.y), (R.x, 2)) + Derivative(phi(R.x, R.y), (R.y, 2)))*cos(2*phi(R.x, R.y)) + epsilon*(Derivative(phi(R.x, R.y), R.x)**2 - Derivative(phi(R.x, R.y), R.y)**2 - 2*Derivative(phi(R.x, R.y), R.x, R.y))*sin(2*phi(R.x, R.y)) + Derivative(phi(R.x, R.y), (R.x, 2)) + Derivative(phi(R.x, R.y), (R.y, 2))

### Now we need to substitute the Ansatz

In [151]:
phi_c = Function('phi_c')(x, y)
theta_1 = Function('theta_1')(x, y)
theta_2 = Function('theta_2')(x, y)
r_1 = Function('r_1')(x, y)
r_2 = Function('r_2')(x, y)

q1, q2 = symbols('q_1 q_2')

ansatz = q1 * theta_1 + q2 * theta_2 + eps * phi_c
display(ansatz)

epsilon*phi_c(R.x, R.y) + q_1*theta_1(R.x, R.y) + q_2*theta_2(R.x, R.y)

In [152]:
euler_lagrange_p = euler_lagrange.subs(phi, ansatz)
euler_lagrange_p = simplify(euler_lagrange_p)
display(euler_lagrange_p)

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*Derivative(theta_1(R.x, R.y), R.x, R.y) + 2*q_2*Derivative(theta_2(R.x, R.y), R.x, R.y) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) + q_1*Derivative(theta_1(R.x, R.y), R.x) + q_2*Derivative(theta_2(R.x, R.y), R.x))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*Derivative(theta_1(R.x, R.y), R.y) + q_2*Derivative(theta_2(R.x, R.y), R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y)) - epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) + q_1*Derivative(theta_1(R.x, R.y), (R.x, 2)) - q_1*Derivative(theta_1(R.x, R.y), (R.y, 2)) + q_2*Derivative(theta_2(R.x, R.y), (R.x, 2)) - q_2*Derivative(theta_2(R.x, R.y), (R.y, 2)) + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) + q_1*Derivative(theta_1(R.x, R.y), R.x) + q_2*Derivative(theta_2(R.x, R.y), R.x))*(epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*Derivative(theta_1(R.x, R.y), R.y) +

In [153]:
euler_lagrange_p = euler_lagrange_p.subs({theta_1.diff(x): -1/r_1 * sin(theta_1),
                                          theta_2.diff(x): -1/r_2 * sin(theta_2),
                                          theta_1.diff(y): 1/r_1 * cos(theta_1),
                                          theta_2.diff(y): 1/r_2 * cos(theta_2)})

In [154]:
display(euler_lagrange_p)

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*Derivative(-sin(theta_1(R.x, R.y))/r_1(R.x, R.y), R.y) + 2*q_2*Derivative(-sin(theta_2(R.x, R.y))/r_2(R.x, R.y), R.y) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y)) - epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) + q_1*Derivative(-sin(theta_1(R.x, R.y))/r_1(R.x, R.y), R.x) - q_1*Derivative(cos(theta_1(R.x, R.y))/r_1(R.x, R.y), R.y) + q_2*Derivative(-sin(theta_2(R.x, R.y))/r_2(R.x, R.y), R.x) - q_2*Derivative(cos(theta_2(R.x, R.y))/r_2(R.x, R.y), R.y) + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))

In [156]:
euler_lagrange_p = euler_lagrange_p.doit()
euler_lagrange_p = euler_lagrange_p.subs({theta_1.diff(x): -1/r_1 * sin(theta_1),
                                          theta_2.diff(x): -1/r_2 * sin(theta_2),
                                          theta_1.diff(y): 1/r_1 * cos(theta_1),
                                          theta_2.diff(y): 1/r_2 * cos(theta_2)})
for arg in euler_lagrange_p.args:
    display(arg)

epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2))

epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2))

q_1*(sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 + sin(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.x)/r_1(R.x, R.y)**2)

q_1*(-sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.y)/r_1(R.x, R.y)**2)

q_2*(sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 + sin(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.x)/r_2(R.x, R.y)**2)

q_2*(-sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.y)/r_2(R.x, R.y)**2)

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*(sin(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.y)/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))**2/r_1(R.x, R.y)**2) + 2*q_2*(sin(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.y)/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))**2/r_2(R.x, R.y)**2) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y))

-epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) - q_1*(-sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.y)/r_1(R.x, R.y)**2) + q_1*(sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 + sin(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.x)/r_1(R.x, R.y)**2) - q_2*(-sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.y)/r_2(R.x, R.y)**2) + q_2*(sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 + sin(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.x)/r_2(R.x, R.y)**2) + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))*(epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)))*cos(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y)

In [157]:
euler_lagrange_p = euler_lagrange_p.subs({r_1.diff(x): cos(theta_1),
                                          r_2.diff(x): cos(theta_2),
                                          r_1.diff(y): sin(theta_1),
                                          r_2.diff(y): sin(theta_2)})

for arg in euler_lagrange_p.args:
    display(arg)

epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2))

epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2))

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*(sin(theta_1(R.x, R.y))**2/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))**2/r_1(R.x, R.y)**2) + 2*q_2*(sin(theta_2(R.x, R.y))**2/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))**2/r_2(R.x, R.y)**2) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y))

-epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) + 4*q_1*sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 + 4*q_2*sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))*(epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)))*cos(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y))

In [161]:
euler_lagrange_p = expand(euler_lagrange_p)
euler_lagrange_p = euler_lagrange_p.subs({eps**2 : 0, eps**3 : 0})
euler_lagrange_p = simplify(euler_lagrange_p)

display(euler_lagrange_p)

epsilon*(-2*q_1*q_2*r_1(R.x, R.y)*r_2(R.x, R.y)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - theta_1(R.x, R.y) - theta_2(R.x, R.y)) + q_1*(2 - q_1)*r_2(R.x, R.y)**2*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - 2*theta_1(R.x, R.y)) + q_2*(2 - q_2)*r_1(R.x, R.y)**2*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - 2*theta_2(R.x, R.y)) + (Derivative(phi_c(R.x, R.y), (R.x, 2)) + Derivative(phi_c(R.x, R.y), (R.y, 2)))*r_1(R.x, R.y)**2*r_2(R.x, R.y)**2)/(r_1(R.x, R.y)**2*r_2(R.x, R.y)**2)